# Ch8. 단위테스트와 리팩토링

[reference](https://www.slideshare.net/hosunglee948/python-52222334)
## 단워 테스트(Unit Test)
- 개발자 뷰
- 함수 단위
- Mock 을 사용
- 빠름
- 코드의 품질 향상

## 기능 테스트(Function Test)
- 사용자 뷰
- 요구사항 단위
- Fixture를 사용
- 느림

# 1. 디자인 원칙과 단위 테스트

- 단위 테스트란 다른 코드의 유효성을 검사하는 코드
- 비즈니스 로직이 특정 조건을 보장하는 지를 확인하기 위해 여러 시나리오를 검증하는 코드
  - 격리:  비즈니스로직에만 집중하고 다른 외부 에이전트와 독립되어야 함(DB, HTTP emd)
  - 성능: 빠르게
  - 자체 검증: 단위 테스트의 실행만으로 결과를 결정할 수 있어야 함

In [5]:
# ut_design_1

import logging
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class MetricsClient:
    """3rd-party metrics client"""

    def send(self, metric_name, metric_value):
        if not isinstance(metric_name, str):
            raise TypeError("expected type str for metric_name")

        if not isinstance(metric_value, str):
            raise TypeError("expected type str for metric_value")

        logger.info("sending %s = %s", metric_name, metric_value)


# class Process:
#     """A job that runs in iterations, and depends on an external object."""

#     def __init__(self):
#         self.client = MetricsClient()  # A 3rd-party metrics client

#     def process_iterations(self, n_iterations):
#         for i in range(n_iterations):
#             result = self.run_process()
#             self.client.send("iteration.{}".format(i), result)

#     def run_process(self):
#         return random.randint(1, 100)


# if __name__ == "__main__":
#     Process().process_iterations(10)


TypeError: expected type str for metric_value

In [6]:
# ut_design_2

class WrappedClient:
    """An object under our control that wraps the 3rd party one."""

    def __init__(self):
        self.client = MetricsClient()

    def send(self, metric_name, metric_value):
        return self.client.send(str(metric_name), str(metric_value))


class Process:
    """Same process, now using a wrapper object."""

    def __init__(self):
        self.client = WrappedClient()

    def process_iterations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send("iteration.{}".format(i), result)

    def run_process(self):
        return random.randint(1, 100)


if __name__ == "__main__":
    Process().process_iterations(10)

INFO:__main__:sending iteration.0 = 62
INFO:__main__:sending iteration.1 = 34
INFO:__main__:sending iteration.2 = 31
INFO:__main__:sending iteration.3 = 59
INFO:__main__:sending iteration.4 = 82
INFO:__main__:sending iteration.5 = 54
INFO:__main__:sending iteration.6 = 37
INFO:__main__:sending iteration.7 = 97
INFO:__main__:sending iteration.8 = 2
INFO:__main__:sending iteration.9 = 17


In [ ]:
#test_ut_design_2

from unittest import TestCase, main
from unittest.mock import Mock

from ut_design_2 import WrappedClient


class TestWrappedClient(TestCase):
    def test_send_converts_types(self):
        wrapped_client = WrappedClient()
        wrapped_client.client = Mock()
        wrapped_client.send("value", 1)

        wrapped_client.client.send.assert_called_with("value", "1")


if __name__ == "__main__":
    main()

## 단위테스트와 소프트웨어 디자인
- 단위테스트는 자기 코드에 동작에 대한 증거이자 안정망
- 테스트의 용이성은 클린 코드의 핵심 가치

# 2. 테스트를 위한 프레임워크와 도구
## unittest
- 표준 라이브러리
- 테스트 시나리오 다룰 때 사용
## pytest
- pip 설치 라이브러리 
- 외부 시스템에 연결하는 등의 의존성이 많은 경우 테스트 케이스를 파라미터화할 수 있는 픽스터(fixture)라는 패치 객체 필요
- 이 때 pytest 사용

In [1]:
from mrstatus import MergeRequestException
from mrstatus import MergeRequestExtendedStatus as MergeRequestStatus

class MergeRequest:
    def __init__(self):
        self._context = {"upvotes": set(), "downvotes": set()}
        self._status = MergeRequestStatus.OPEN

    def close(self):
        self._status = MergeRequestStatus.CLOSED

    @property
    def status(self):
        if self._status == MergeRequestStatus.CLOSED:
            return self._status

        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING

    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("can't vote on a closed merge request")

    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

In [3]:
from mrstatus import MergeRequestException
from mrstatus import MergeRequestExtendedStatus as MergeRequestStatus


class MergeRequest:
    def __init__(self):
        self._context = {"upvotes": set(), "downvotes": set()}
        self._status = MergeRequestStatus.OPEN

    def close(self):
        self._status = MergeRequestStatus.CLOSED

    @property
    def status(self):
        if self._status == MergeRequestStatus.CLOSED:
            return self._status

        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING

    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("can't vote on a closed merge request")

    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

In [4]:
class ExtendedCases:
    """For the MRs that use the extended status."""

    def test_cannot_upvote_on_closed_merge_request(self):
        self.merge_request.close()
        self.assertRaises(
            MergeRequestException, self.merge_request.upvote, "dev1"
        )

    def test_cannot_downvote_on_closed_merge_request(self):
        self.merge_request.close()
        self.assertRaisesRegex(
            MergeRequestException,
            "can't vote on a closed merge request",
            self.merge_request.downvote,
            "dev1",
        )

## pytest
- assert 구문을 사용해 조건을 검사하는 것이 가능
- 재사용 가능한 기능을 쉽게 만들 수 있음

### 픽스처(Fixture)
- 테스트 사전/사후에 사용 가능한 리소스 또는 모듈
- `@pytest.fixture`  

## 코드 커버리지
- test runner는 pip를 통해 설치 가능한 커버리지 플러그인을 제공
- 커버리지 플러그인은 코드의 어떤 부분이 실행되었는지 알려줌
- converage 라이브러리
- pytest-cov

```
$ pytest \
> --cov-report term-missing \
> --cov=coverage_1 \
> test_coverage_1.py 
================== test session starts ==================
platform win32 -- Python 3.9.7, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: C:\Users\ssafy_eunseong\Desktop\CleanCodeinPython_Code\Chapter08
plugins: anyio-3.3.4, Faker-9.3.1, cov-3.0.0
collected 16 items

test_coverage_1.py ................                [100%]

----------- coverage: platform win32, python 3.9.7-final-0 -----------
Name            Stmts   Miss  Cover   Missing
---------------------------------------------
coverage_1.py      39      1    97%   44
---------------------------------------------
TOTAL              39      1    97%


================== 16 passed in 0.39s ==================

높은 커버리지만으로 해당 코드에 대해 품질을 보증할 수 없음

## 모의(Mock) 객체
- 원하지 않는 부작용으로부터 테스트 코드를 보호하는 가장 좋은 방법 중 하나 
